In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 21:00:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.version

'3.5.1'

In [3]:
df_fhv = spark.read \
    .option("header", "true") \
    .csv('data/raw/fhv/2019/10/')

df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [4]:
import pandas as pd

df_fhv_pd = pd.read_csv('data/raw/fhv/2019/10/fhv_tripdata_2019-10.csv.gz', nrows=1000)

spark.createDataFrame(df_fhv_pd).schema

/Users/saschapogadaev/miniconda3/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [5]:
df_fhv_pd.sample()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
713,B01730,2019-10-01 00:36:47,2019-10-01 00:53:00,264.0,196.0,NaN,B02096


In [6]:
from pyspark.sql import types

fhv_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PUlocationID', types.IntegerType(), True),
    types.StructField('DOlocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.DoubleType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [7]:
df_fhv = spark.read \
    .option('header', 'true') \
    .schema(fhv_schema) \
    .csv('data/raw/fhv/2019/10/fhv_tripdata_2019-10.csv.gz')

df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [8]:
df_fhv.repartition(6).write.parquet('data/pq/')

24/03/06 20:49:33 ERROR Inbox: Ignoring error                       (0 + 1) / 1]
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apach

24/03/06 20:49:43 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928)

In [3]:
df_fhv = spark.read.parquet('data/pq/*')

In [4]:
df_fhv.registerTempTable('trips_data')

/Users/saschapogadaev/miniconda3/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_fhv.show()


+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01436|2019-10-23 00:41:45|2019-10-23 01:00:36|         264|         226|   NULL|                B01436|
|              B01626|2019-10-13 19:39:37|2019-10-13 19:53:06|         264|         205|   NULL|                B01626|
|              B01553|2019-10-12 14:09:28|2019-10-12 14:23:04|         264|         196|   NULL|                B01553|
|              B02677|2019-10-19 12:42:15|2019-10-19 12:45:01|         264|          76|   NULL|                B02677|
|              B00727|2019-10-15 23:20:58|2019-10-15 23:32:59|         264|         264|   NULL|                B00727|
|              B00310|2019-10-05 17:41:1

In [7]:
spark.sql("""
SELECT
    date_trunc('day', pickup_datetime) AS day,
    count(1)
FROM
    trips_data
GROUP BY 
    1
ORDER BY
    1
""").show()

+-------------------+--------+
|                day|count(1)|
+-------------------+--------+
|2019-10-01 00:00:00|   59873|
|2019-10-02 00:00:00|   68746|
|2019-10-03 00:00:00|   71638|
|2019-10-04 00:00:00|   68227|
|2019-10-05 00:00:00|   52398|
|2019-10-06 00:00:00|   45665|
|2019-10-07 00:00:00|   66137|
|2019-10-08 00:00:00|   64049|
|2019-10-09 00:00:00|   60468|
|2019-10-10 00:00:00|   68559|
|2019-10-11 00:00:00|   67715|
|2019-10-12 00:00:00|   51434|
|2019-10-13 00:00:00|   45900|
|2019-10-14 00:00:00|   52665|
|2019-10-15 00:00:00|   62610|
|2019-10-16 00:00:00|   68156|
|2019-10-17 00:00:00|   67656|
|2019-10-18 00:00:00|   68471|
|2019-10-19 00:00:00|   52530|
|2019-10-20 00:00:00|   48304|
+-------------------+--------+
only showing top 20 rows



In [13]:
spark.sql("""
SELECT
    MAX(TIMESTAMPDIFF(HOUR, pickup_datetime, dropOff_datetime))
FROM
    trips_data
""").show()

+-----------------------------------------------------------+
|max(timestampdiff(HOUR, pickup_datetime, dropOff_datetime))|
+-----------------------------------------------------------+
|                                                     631152|
+-----------------------------------------------------------+



In [25]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('data/raw/zones/taxi_zone_lookup.csv')

In [26]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
import pandas as pd

df_zones = pd.read_csv('data/raw/zones/taxi_zone_lookup.csv', nrows=1000)

spark.createDataFrame(df_zones).schema

/Users/saschapogadaev/miniconda3/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


StructType([StructField('LocationID', LongType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [30]:
from pyspark.sql import types

zones_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [31]:
df_zones = spark.read \
    .option('header', 'true') \
    .schema(zones_schema) \
    .csv('data/raw/zones/taxi_zone_lookup.csv')

df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [47]:
df_result = df_fhv.join(df_zones, df_fhv.PUlocationID == df_zones.LocationID)
df_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+-------------+--------------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|LocationID|      Borough|                Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+-------------+--------------------+------------+
|              B01436|2019-10-23 00:41:45|2019-10-23 01:00:36|         264|         226|   NULL|                B01436|       264|      Unknown|                  NV|         N/A|
|              B01626|2019-10-13 19:39:37|2019-10-13 19:53:06|         264|         205|   NULL|                B01626|       264|      Unknown|                  NV|         N/A|
|              B01553|2019-10-12 14:09:28|2019-10-12 14:23:04|         264|         196|   NULL|         

In [43]:
df_result.registerTempTable('trip_result')


/Users/saschapogadaev/miniconda3/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [55]:
spark.sql("""
SELECT
    Zone,
    count(1)
FROM
    trip_result
GROUP BY
    1
ORDER BY
    2
""").show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
|        Battery Park|      15|
|Saint Michaels Ce...|      23|
|Breezy Point/Fort...|      25|
|Marine Park/Floyd...|      26|
|        Astoria Park|      29|
|    Inwood Hill Park|      39|
|       Willets Point|      47|
|Forest Park/Highl...|      53|
|  Brooklyn Navy Yard|      57|
|        Crotona Park|      62|
|        Country Club|      77|
|     Freshkills Park|      89|
|       Prospect Park|      98|
|     Columbia Street|     105|
|  South Williamsburg|     110|
+--------------------+--------+
only showing top 20 rows

